In [29]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
import pickle
from category_encoders import TargetEncoder
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
import time
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor


In [2]:
cleanData = pd.read_csv('./data csv/kijkcijfersWeer.csv')
cleanData.sample(5)

,Unnamed: 0,FullDate,date,hour,Kanaal,Programma,Lengte_sec,Kijkers,Temperatuur,Gevoelstemp,Regen,Sneeuw,Weercode,Bewolking,Windsnelheid,Zonnenschijn,isFeestdag,Weekdag,isWeekend,Seizoen
36215,36215,2021-10-08 20:35:18,2021-10-08,20,PLAY5,TEMPTATION ISLAND - LOVE OR LEAVE,2961,138754,11.9,10.6,0.0,0.0,0,1,8.9,0.0,0,4,0,herfst
31712,31712,2021-02-25 22:00:17,2021-02-25,22,VTM,TELEFACTS NU,2602,264207,7.7,5.6,0.0,0.0,1,41,9.0,0.0,0,3,0,winter
26975,26975,2020-06-17 20:36:26,2020-06-17,20,Canvas,DE AFSPRAAK,2683,234316,16.4,16.9,1.1,0.0,55,100,9.4,0.0,0,2,0,lente
8489,8489,2017-12-03 19:56:18,2017-12-03,19,VTM,GE HADT ERBIJ MOETEN ZIJN,1404,378203,4.0,1.1,0.0,0.0,3,100,10.0,0.0,0,6,1,herfst
12844,12844,2018-07-10 13:00:05,2018-07-10,13,EEN,HET 1 UUR-JOURNAAL,1629,396735,17.5,14.1,0.0,0.0,3,100,21.3,0.0,0,1,0,zomer


Feature engineering

In [3]:
def lagFeatures(df, n):
  for i in range(1,n+1):
    df[f'KijkersLag{i}'] = df.sort_values('FullDate').groupby('Programma')['Kijkers'].shift(i).fillna(df.groupby('Programma')['Kijkers'].transform('mean'))
  return df

In [4]:
cleanData = lagFeatures(cleanData, 3)
cleanData[cleanData['Programma'] == 'THUIS'][['FullDate', 'Kijkers', 'KijkersLag1', 'KijkersLag2', 'KijkersLag3']]


,FullDate,Kijkers,KijkersLag1,KijkersLag2,KijkersLag3
40,2016-10-03 20:14:23,1268561,1.059928e+06,1.059928e+06,1.059928e+06
60,2016-10-04 20:09:27,1169791,1.268561e+06,1.059928e+06,1.059928e+06
80,2016-10-05 20:13:52,1244502,1.169791e+06,1.268561e+06,1.059928e+06
121,2016-10-07 19:57:34,1156477,1.244502e+06,1.169791e+06,1.268561e+06
180,2016-10-10 19:59:00,1315826,1.156477e+06,1.244502e+06,1.169791e+06
...,...,...,...,...,...
61265,2025-04-04 20:24:13,779244,1.018053e+06,9.942260e+05,1.066599e+06
61325,2025-04-07 20:14:35,869193,7.792440e+05,1.018053e+06,9.942260e+05
61345,2025-04-08 20:13:09,880767,8.691930e+05,7.792440e+05,1.018053e+06
61366,2025-04-09 20:19:53,802132,8.807670e+05,8.691930e+05,7.792440e+05


Kardinaliteiten

In [5]:
#kardinaliteiten voor betere toepassingen
for kolom in cleanData.columns:
  print(f'{kolom}: {cleanData[kolom].nunique()}')

Unnamed: 0: 61405
FullDate: 61318
date: 3051
hour: 21
Kanaal: 29
Programma: 5963
Lengte_sec: 6774
Kijkers: 58860
Temperatuur: 414
Gevoelstemp: 478
Regen: 62
Sneeuw: 25
Weercode: 13
Bewolking: 101
Windsnelheid: 456
Zonnenschijn: 3555
isFeestdag: 2
Weekdag: 7
isWeekend: 2
Seizoen: 4
KijkersLag1: 58998
KijkersLag2: 56714
KijkersLag3: 55143


One Hot Encoding

In [6]:
#one hot encoding voor lage cardinaliteit
oneHotEnc = OneHotEncoder(handle_unknown="ignore")
lageKardinaliteit = cleanData[[ 'hour','Kanaal', 'isFeestdag', 'Weekdag', 'Seizoen']]
oneHot = oneHotEnc.fit_transform(lageKardinaliteit)
dfOneHot = pd.DataFrame(oneHot.toarray(), 
                            columns=oneHotEnc.get_feature_names_out(), 
                            index=lageKardinaliteit.index)

cleanData = cleanData.drop(columns=['hour', 'Kanaal', 'isFeestdag', 'Weekdag', 'Seizoen'])
cleanData = pd.concat([cleanData, dfOneHot], axis=1)

cleanData.sample(5)

,Unnamed: 0,FullDate,date,Programma,Lengte_sec,Kijkers,Temperatuur,Gevoelstemp,Regen,Sneeuw,...,Weekdag_1,Weekdag_2,Weekdag_3,Weekdag_4,Weekdag_5,Weekdag_6,Seizoen_herfst,Seizoen_lente,Seizoen_winter,Seizoen_zomer
45023,45024,2022-12-26 16:45:59,2022-12-26,BRITAIN'S BUSIEST AIRPORT HEATHROW,1364,215602,6.6,2.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21618,21618,2019-09-24 20:15:08,2019-09-24,THUIS,1526,1136071,15.1,14.4,0.2,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
46978,46980,2023-04-03 22:07:00,2023-04-03,DE IDEALE WERELD,2101,266727,4.7,-0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6097,6097,2017-08-05 22:41:10,2017-08-05,ROAD HOUSE,6342,125830,14.8,14.5,0.2,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
50077,50079,2023-09-06 18:25:31,2023-09-06,LISA,1330,314046,29.0,30.2,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
#opslaan bij models
modelsDir = os.getcwd()
modelFile = os.path.join(modelsDir, ".",
                          "models", "oneHotEncoder.pkl")
os.makedirs(os.path.dirname(modelFile), exist_ok=True)
with open(modelFile, 'wb') as m:
  pickle.dump(oneHotEnc, m)

Target Encoding

In [8]:
#target encoding voor medium kardinaliteiten
targetEncoding = TargetEncoder()
medKardinaliteit = cleanData[['date', 'Programma', 'Lengte_sec', 'Temperatuur', 'Gevoelstemp', 'Regen', 'Bewolking', 'Windsnelheid', 'Zonnenschijn']]
#verdere feature engineering op vorig model
target = targetEncoding.fit_transform(medKardinaliteit, cleanData['Kijkers'])
target.sample(10)

,date,Programma,Lengte_sec,Temperatuur,Gevoelstemp,Regen,Bewolking,Windsnelheid,Zonnenschijn
39553,424835.581061,3.229613e+05,1465,17.8,14.7,0.0,0,5.8,3600.00
26539,439026.698960,5.039425e+05,2195,18.0,13.9,0.0,2,19.5,3600.00
16261,454846.723960,7.918627e+05,1428,3.6,0.6,0.0,100,9.9,0.00
60465,439943.548960,1.035436e+06,2883,12.2,8.5,0.0,100,18.8,0.00
30597,529251.523960,4.218637e+05,4883,1.1,-2.2,0.1,29,8.2,0.00
19144,460031.223960,6.386436e+05,1633,15.4,12.9,0.0,91,16.9,3600.00
6704,466994.373960,6.386436e+05,1631,21.5,21.1,0.0,100,15.6,1117.36
18362,426637.323960,6.170952e+05,1474,6.5,3.0,0.0,69,7.4,55.56
50163,408860.798960,3.691028e+05,2875,25.4,27.1,0.0,21,8.5,0.00
28777,480483.273960,1.908959e+05,2325,16.2,15.2,0.0,99,12.8,0.00


In [9]:
cleanData = cleanData.drop(columns=['date', 'Programma', 'Lengte_sec', 'Temperatuur', 'Gevoelstemp', 'Regen', 'Bewolking', 'Windsnelheid', 'Zonnenschijn'])
cleanData = pd.concat([cleanData, target], axis=1)
cleanData.sample(5)

,Unnamed: 0,FullDate,Kijkers,Sneeuw,Weercode,isWeekend,KijkersLag1,KijkersLag2,KijkersLag3,hour_0,...,Seizoen_zomer,date,Programma,Lengte_sec,Temperatuur,Gevoelstemp,Regen,Bewolking,Windsnelheid,Zonnenschijn
2023,2023,2017-01-13 20:36:00,779247,0.0,51,0,675591.0,580134.0,579917.0,0.0,...,0.0,478971.87396,463071.762833,5464,4.4,-2.2,0.3,100,33.2,0.0
37344,37344,2021-12-04 19:44:58,487883,0.0,0,1,937578.0,1032279.0,1087489.0,0.0,...,0.0,428041.82396,791862.682708,1820,4.8,-0.0,0.0,1,19.5,0.0
43978,43979,2022-11-02 17:02:16,242046,0.0,3,0,317633.0,225285.0,134208.0,0.0,...,0.0,423452.14896,311475.622047,3355,13.4,9.5,0.0,100,21.5,3600.0
36452,36452,2021-10-20 21:48:10,265753,0.0,3,0,324312.0,431820.0,330767.0,0.0,...,0.0,470427.64896,360157.222324,3118,14.6,12.2,0.0,95,19.6,0.0
55409,55412,2024-06-04 20:08:28,539271,0.0,3,0,611890.0,523961.0,493559.0,0.0,...,0.0,416450.07396,617095.220955,1514,18.4,16.8,0.0,88,17.4,3600.0


Verbanden

In [10]:
dataOneHotTarget = cleanData.select_dtypes(include=[np.number])

verband = dataOneHotTarget.corr()

verband = verband['Kijkers'].abs().sort_values(ascending=False)

print(f'{verband.head(10)}')
dataOneHotTarget.shape

Kijkers        1.000000
KijkersLag1    0.894582
KijkersLag2    0.893195
KijkersLag3    0.875979
Programma      0.827215
hour_19        0.386184
Kanaal_EEN     0.333420
date           0.333361
hour_22        0.184308
hour_17        0.176407
Name: Kijkers, dtype: float64


(61405, 80)

In [11]:
# Eerst omzetten naar strings, daarna komma's verwijderen, en uiteindelijk naar floats
for col in ['KijkersLag1', 'KijkersLag2', 'KijkersLag3', 'Programma']:
    cleanData[col] = cleanData[col].astype(str).str.replace(',', '').astype(float)

# Print het resultaat
print(cleanData[['Kijkers', 'KijkersLag1', 'KijkersLag2', 'KijkersLag3', 'Programma']].head(10))


   Kijkers    KijkersLag1    KijkersLag2    KijkersLag3      Programma
0   721850  892001.072096  892001.072096  892001.072096  892001.072096
1   709606  628487.089971  628487.089971  628487.089971  628487.089971
2   548239  478421.179487  478421.179487  478421.179487  474186.638952
3   523610  791862.682708  791862.682708  791862.682708  791862.682708
4   496216  491224.740741  491224.740741  491224.740741  491217.906057
5   447427  351454.000000  351454.000000  351454.000000  431290.191170
6   424041  585853.225914  585853.225914  585853.225914  585853.225914
7   369066  408691.165611  408691.165611  408691.165611  408691.165611
8   368549  336230.500000  336230.500000  336230.500000  424185.509591
9   360544  332870.588235  332870.588235  332870.588235  332901.616764


In [22]:
#opslaan bij models
targetModelDir = os.getcwd()
targetModelPath = os.path.join(targetModelDir, ".",
                          "models", "oneHotEncoder.pkl")
os.makedirs(os.path.dirname(targetModelPath), exist_ok=True)
with open(targetModelPath, 'wb') as m:
  pickle.dump(oneHotEnc, m)

Model tests

In [40]:
def test_models(models, X, y, cv=10, scoring='neg_mean_absolute_error'):
    res = {}
    for m in models:
        model = type(m).__name__
        scores = cross_val_score(m, X, y, cv=cv, scoring=scoring)
        res[model] = {
            'scores': scores,
            'mean_score': scores.mean()
        }
        print(f"Scores {model}\n{scores}\nMAE = {scores.mean()}")
    return res

Lasso Regression

In [ ]:
models = [
    RandomForestRegressor(verbose=True), Lasso()
]

# Zorg ervoor dat X en y goed geschaald zijn
scaler = StandardScaler()
X = dataOneHotTarget.drop(columns=['Kijkers', 'Unnamed: 0'])
X_scaled = scaler.fit_transform(X)
y = dataOneHotTarget['Kijkers']

# Run de test
results = test_models(models, X_scaled, y)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.4min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.0min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.5min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.3min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.0min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.2min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.5min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.4min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Do

Scores RandomForestRegressor
[-56340.67393584 -52667.80964664 -50883.93049992 -48159.0719378
 -60235.06581013 -51754.82341205 -50855.67859609 -49048.17195114
 -50496.49685668 -50382.99843322]
MAE = -52082.47210795064


c:\Users\krist\Documents\Hogent_IT\2de_jaar\Machine_Learning\ML_project_kijkcijfers\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.042e+13, tolerance: 4.190e+11
  model = cd_fast.enet_coordinate_descent(
c:\Users\krist\Documents\Hogent_IT\2de_jaar\Machine_Learning\ML_project_kijkcijfers\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.104e+13, tolerance: 4.223e+11
  model = cd_fast.enet_coordinate_descent(
c:\Users\krist\Documents\Hogent_IT\2de_jaar\Machine_Learning\ML_project_kijkcijfers\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: Convergen

Scores Lasso
[-70240.31538707 -67302.89346697 -64776.0347236  -62745.39174351
 -69298.51653763 -64885.35384331 -63986.35690782 -58469.83377457
 -62217.9159027  -62300.31546391]
MAE = -64622.292775108515


c:\Users\krist\Documents\Hogent_IT\2de_jaar\Machine_Learning\ML_project_kijkcijfers\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.101e+13, tolerance: 4.341e+11
  model = cd_fast.enet_coordinate_descent(


RandomForest heeft betere scores:
  
  RandomForest
  MAE = -52082.47210795064

  Lasso
  MAE = -64622.292775108515

Naar csv schrijven

In [23]:
dir = os.getcwd()
pathFile = os.path.join(dir, "./data csv/oneHotTarget.csv")

os.makedirs(os.path.dirname(pathFile), exist_ok=True)

dataOneHotTarget.to_csv(pathFile)


keuze maken voor model

In [53]:
#trainset en testset maken
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

def scoreModel(model,X,y, cv=5):
    print(f"Model: {model.__class__.__name__}")
    start = time.time()
        
    # MAE
    mae_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    mae = -np.mean(mae_scores)

    # MAPE
    mape_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_percentage_error')
    mape = -np.mean(mape_scores) * 100

    duration = time.time() - start
    min, sec = divmod(duration, 60)

    print(f"MAE:  {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"Tijd: {int(min)}m {sec:.1f}s")

    return {'Model': model.__class__.__name__, 'MAE': mae, 'MAPE': mape, 'Tijd': duration}
def startModel(models, X, y, cv=5):
    results = []
    for model in models:
        result = scoreModel(model, X, y, cv=cv)
        results.append(result)
    return results

In [55]:
linear = LinearRegression()
decision = DecisionTreeRegressor()
randomForest = RandomForestRegressor()
gradientBoosting = GradientBoostingRegressor()
xgb = XGBRegressor()
extraTrees = ExtraTreesRegressor()
histGB = HistGradientBoostingRegressor()

startModel([linear, decision, randomForest, gradientBoosting, xgb, extraTrees, histGB], X, y)


Model: LinearRegression
MAE:  65143.6876
MAPE: 20.22%
Tijd: 0m 2.2s
Model: DecisionTreeRegressor
MAE:  78011.1780
MAPE: 22.25%
Tijd: 0m 18.7s
Model: RandomForestRegressor
MAE:  53339.9060
MAPE: 16.10%
Tijd: 18m 43.7s
Model: GradientBoostingRegressor
MAE:  57059.1948
MAPE: 17.32%
Tijd: 4m 15.8s
Model: XGBRegressor
MAE:  53561.6539
MAPE: 15.62%
Tijd: 0m 4.2s
Model: ExtraTreesRegressor
MAE:  51912.1192
MAPE: 15.50%
Tijd: 11m 29.4s
Model: HistGradientBoostingRegressor
MAE:  52753.3521
MAPE: 15.64%
Tijd: 0m 11.3s


[{'Model': 'LinearRegression',
  'MAE': np.float64(65143.68759504148),
  'MAPE': np.float64(20.218955732563145),
  'Tijd': 2.2159063816070557},
 {'Model': 'DecisionTreeRegressor',
  'MAE': np.float64(78011.17796596368),
  'MAPE': np.float64(22.25006578499576),
  'Tijd': 18.742029428482056},
 {'Model': 'RandomForestRegressor',
  'MAE': np.float64(53339.90603240779),
  'MAPE': np.float64(16.095086679558193),
  'Tijd': 1123.6802473068237},
 {'Model': 'GradientBoostingRegressor',
  'MAE': np.float64(57059.1948182157),
  'MAPE': np.float64(17.321027485391024),
  'Tijd': 255.79759216308594},
 {'Model': 'XGBRegressor',
  'MAE': np.float64(53561.65390625),
  'MAPE': np.float64(15.621936023235323),
  'Tijd': 4.17343282699585},
 {'Model': 'ExtraTreesRegressor',
  'MAE': np.float64(51912.11915055777),
  'MAPE': np.float64(15.498918452099291),
  'Tijd': 689.3694071769714},
 {'Model': 'HistGradientBoostingRegressor',
  'MAE': np.float64(52753.35208368045),
  'MAPE': np.float64(15.638216848550183),


ExtraTreesRegressor heeft beste MAE en MAPE